# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['data-modeling-with-apache-cassandra-cassandra-1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [24]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Since we will use `session_id` and `item_in_session` in the where clause, the primary key should consist of `session_id` and `item_in_session`. Also, `session_id` is the larger unit, it should come first when considering partitions. When `session_id`s are the same, a composite key of these two columns is sufficient since rows can be distinguished by `item_in_ssession`. 

In [26]:
query = "CREATE TABLE IF NOT EXISTS music_playlist "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [27]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_playlist (session_id, item_in_session, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [28]:
query = "SELECT artist, song, length FROM music_playlist WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(columns=['artist', 'song', 'length'])
for row in rows:
    df.loc[len(df)] = [row.artist, row.song, row.length]
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Since we will use `user_id` and `session_id` in the where clause, the primary key should consist of `user_id` and `session_id`. When we consider the hierarchy of the column, `user_id` should come first then `session_id`. To distinguish the rows with the same `user_id` and `session_id`, we must add `item_in_session` as a primary key. However, it should be a clustering key to order the results.

In [29]:
query = "CREATE TABLE IF NOT EXISTS music_user_playlist "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [30]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_user_playlist (user_id, session_id, item_in_session, artist, song, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [31]:
query = "SELECT artist, song, first_name, last_name FROM music_user_playlist WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(columns=['artist', 'song', 'first_name', 'last_name'])
for row in rows:
    df.loc[len(df)] = [row.artist, row.song, row.first_name, row.last_name]    
df

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Since we will use `song` in the where clause, the primary key should be `song`. To distinguish rows with the same `song` title, we need to add one more column, `user_id`, to the primary key. Also, we will not specify the `user_id` in the where clause; the `user_id` should come at the end of the composite key list.

In [32]:
query = "CREATE TABLE IF NOT EXISTS user_history "
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
# query = query + "(song text, user_id int, first_name text, last_name text, session_id int, item_in_session int, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)           

In [34]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history (song, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [35]:
query = "SELECT first_name, last_name FROM user_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(columns=['first_name', 'last_name'])
for row in rows:
    df.loc[len(df)] = [row.first_name, row.last_name]
df

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


## Drop the tables before closing out the sessions

In [36]:
query = "DROP TABLE music_playlist"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE music_user_playlist"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Close the session and cluster connection¶

In [37]:
session.shutdown()
cluster.shutdown()